In [1]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

loader = PyPDFLoader("../data/Sophia_Resume.pdf")
pages = loader.load_and_split()
userName="Sophia"

In [2]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

In [3]:

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())
#docs = faiss_index.similarity_search("How will the community be engaged?", k=2)
#for doc in docs:
#    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

In [4]:
retriever = faiss_index.as_retriever()

    #prompt = ChatPromptTemplate.from_template(template)
   
prompt = hub.pull("rlm/rag-prompt")
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
        return "\n\n".join([d.page_content for d in docs])
    
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

solution=chain.invoke(
        f"""
        Structure {userName} background by filling in the section below
        - Best Matching Job Positions (for example "Data Engineer", "Product Manager", etc) rank tron best fit to less fit:
        - Top Skills he/she has (10 to 20):
        - Industries (tike Retail, Tech, Consulting, Government, Finance, etc.):
        - Years of Experience (estimate if not specified, e.g 3 YOE, 5+ YOE):
        - Location (City, Country):
        - Open to Relocate (Yes/No):
        - Full Experience (including everything):
        """)

print(solution)
    


- Best Matching Job Positions: Fullstack Software Developer, Software Developer Intern, Embedded Software Developer
- Top Skills: React, JavaScript, NodeJS, MongoDB, SQL, Git, RESTful API, Test-driven development, Agile methodology, AWS
- Industries: Tech
- Years of Experience: 1-2 YOE
- Location: Toronto, Canada
- Open to Relocate: No
